In [1]:
import setup
setup.init_django()

/home/nazk33r/Documents/LLM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from blog.models import BlogPost
from blog import services
from decouple import config

In [3]:
qs = BlogPost.objects.filter(can_delete=True)
qs

<QuerySet [<BlogPost: BlogPost object (2)>]>

In [4]:
from decouple import config
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from llama_index.core import Settings
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import requests
import json

In [5]:
# import psycopg2

# connection_string = "postgresql://postgres:z@localhost:5432"
# db_name = "vector_db"
# conn = psycopg2.connect(connection_string)
# conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")

In [6]:
HUGGINGFACE_API_KEY = config("HUGGINGFACE_API_KEY")

In [7]:
vector_db_name = 'vector_db'
vector_db_table_name = 'blogpost'

In [8]:
LLM_MODEL = config("LLM_MODEL", default='gpt2')
EMBEDDING_MODEL = config("EMBEDDING_MODEL", default='sentence-transformers/all-MiniLM-L6-v2')
EMBEDDING_LENGTH = config("EMBEDDING_LENGHT", default=1536, cast=int)
HUGGINGFACE_API_KEY = config("HUGGINGFACE_API_KEY")


In [9]:
headers = {
    "Authorization": f"Bearer {HUGGINGFACE_API_KEY}"
}

In [10]:
def get_embedding(text, model=EMBEDDING_MODEL):
    api_url = f"https://api-inference.huggingface.co/models/{model}"
    payload = {
        "inputs": text,
        "options": {"wait_for_model": True}
    }
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()[0]['embedding']


In [11]:
from sqlalchemy import make_url
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
DATABASE_URL = config('DATABASE_URL')


In [12]:
url = make_url(DATABASE_URL)

# Create VectorStore
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=config("VECTOR_DB_TABLE_NAME"),
    embed_dim=EMBEDDING_LENGTH  # embedding dimension
)

In [13]:
# Storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Index and query engine
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context
)
query_engine = index.as_query_engine()


In [14]:
# Define the LLM model settings
class CustomLLM:
    def __init__(self, model_name):
        self.model_name = model_name

    def __call__(self, prompt):
        # Define how to generate text using the model, if needed
        pass

# Settings
llm_model_name = config("LLM_MODEL", default='gpt2')
custom_llm = CustomLLM(model_name=llm_model_name)

Settings.llm = custom_llm
Settings.embed_model = get_embedding

In [15]:
query_engine.query("My query")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}